In [27]:
from average_decision_ordering import calc_ado
import numpy as np
import pandas as pd
from itertools import product
from tqdm import tqdm
import pathlib
path = pathlib.Path.cwd()

In [40]:
results_path = path / "results" / "bootstrap" 
df = pd.DataFrame()
for pred_file in tqdm(list(results_path.rglob("ll_predictions_0.npy"))):
    deets = str(pred_file).split("/")
    label = f"{deets[-4]}_{deets[-3]}"
    data = pd.DataFrame({label: np.hstack(np.load(pred_file))})
    df = pd.concat([df, data], axis=1)

100%|██████████| 15/15 [00:06<00:00,  2.28it/s]


ValueError: DataFrame constructor not properly called!

In [47]:
ys = pd.DataFrame()
for rinv in ("0p0", "0p3", "1p0"):
    dfy = pd.read_hdf(path.parent / "data" / "HL" / f"HL-{rinv}.h5", "targets").iloc[:290880].rename(columns={'targets': rinv})
    ys = pd.concat([ys, dfy], axis=1)
print(df)
print(ys)

           CNN_1_0p3     CNN_1_1p0  CNN_1_0p0  CNN_2_0p3     CNN_2_1p0  \
0       9.254444e-10  9.964536e-01   0.006464   0.000053  9.968291e-01   
1       9.917690e-04  3.986901e-08   0.999995   0.001512  9.447006e-06   
2       7.398299e-03  9.977360e-01   0.999940   0.000070  9.969307e-01   
3       9.926291e-01  9.999413e-01   0.422683   0.996739  9.973003e-01   
4       9.999328e-01  9.990017e-01   0.000436   0.999895  9.850632e-01   
...              ...           ...        ...        ...           ...   
290875  2.071532e-04  1.160450e-01   0.992490   0.000002  3.623620e-04   
290876  9.979455e-01  9.998430e-01   0.009248   0.999794  9.999913e-01   
290877  9.793763e-01  1.000000e+00   0.999936   0.995574  9.999987e-01   
290878  1.901706e-06  2.851932e-03   0.000227   0.000005  3.730778e-10   
290879  9.997408e-01  9.999998e-01   0.215365   0.998504  9.999993e-01   

           CNN_2_0p0   PFN_0p3   PFN_1p0   PFN_0p0   EFN_0p3   EFN_1p0  \
0       6.484079e-05  0.002022  1.000

In [57]:
labels = list(df.columns)
pairs = list(product(labels, labels))
rinvs = ["0p0", "0p3", "1p0"]
for rinv in rinvs:
    for p1, p2 in pairs:
        if rinv in p1 and rinv in p2:
            if "PFN" in p1 or "PFN" in p2:
                d1 = df[p1].values
                d2 = df[p2].values
                y = ys[rinv].values
                ado_val = calc_ado(d1, d2, y, 500000)
                print(p1, p2, ado_val)
            else:
                pass

CNN_1_0p0 PFN_0p0 0.9954021
CNN_2_0p0 PFN_0p0 0.9964838
PFN_0p0 CNN_1_0p0 0.99533844
PFN_0p0 CNN_2_0p0 0.99638057
PFN_0p0 PFN_0p0 0.99999917
PFN_0p0 EFN_0p0 0.99652594
PFN_0p0 HL_0p0 0.9845223
EFN_0p0 PFN_0p0 0.99631685
HL_0p0 PFN_0p0 0.98487246
CNN_1_0p3 PFN_0p3 0.98850024
CNN_2_0p3 PFN_0p3 0.99142784
PFN_0p3 CNN_1_0p3 0.9886662
PFN_0p3 CNN_2_0p3 0.99132293
PFN_0p3 PFN_0p3 0.99999917
PFN_0p3 EFN_0p3 0.9942135
PFN_0p3 HL_0p3 0.97852093
EFN_0p3 PFN_0p3 0.99421614
HL_0p3 PFN_0p3 0.97844183
CNN_1_1p0 PFN_1p0 0.9977104
CNN_2_1p0 PFN_1p0 0.997841
PFN_1p0 CNN_1_1p0 0.9976502
PFN_1p0 CNN_2_1p0 0.9977774
PFN_1p0 PFN_1p0 1.0
PFN_1p0 EFN_1p0 0.99840313
PFN_1p0 HL_1p0 0.99951696
EFN_1p0 PFN_1p0 0.9983825
HL_1p0 PFN_1p0 0.99950063
